In [91]:
#%matplotlib notebook

In [1]:
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [3]:
from netCDF4 import Dataset

In [4]:
import glob

In [5]:
import re

In [6]:
import datetime

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet

In [8]:
def hvel(r):
    x=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    return (bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**x

In [9]:
def pres(r):
    return pch+dph*np.exp(-(rmaxh/r)**bh)

In [10]:
def hvel2d(l1,l2,vtx,vty):
    r=np.sqrt(l1**2+l2**2)
    xh=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    ur=(bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**xh
    theta=np.arctan2(l2,l1)
    ux=-ur*np.sin(theta)
    uy=ur*np.cos(theta)
    return ux+vtx,uy+vty, pres(r)

In [14]:
path='BestTrack/'

In [15]:
data=pd.read_csv(path+'Allstorms.ibtracs_all.v03r09.csv', header=1, low_memory=False)

In [16]:
data.head()

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
0,NaN,Year,#,BB,BB,NaN,YYYY-MM-DD HH:MM:SS,NaN,deg_north,deg_east,...,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile
1,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 06:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
2,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 12:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
3,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-25 18:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
4,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-26 00:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000


In [17]:
storms=Dataset(path+'Allstorms.ibtracs_all.v03r09.nc')

In [18]:
storms.variables.keys()

[u'storm_sn',
 u'name',
 u'numObs',
 u'season',
 u'track_type',
 u'genesis_basin',
 u'num_basins',
 u'basin',
 u'wind_avg_period',
 u'source',
 u'source_time',
 u'lat_for_mapping',
 u'lon_for_mapping',
 u'nature_for_mapping',
 u'source_lat',
 u'source_lon',
 u'source_wind',
 u'source_pres',
 u'dist2land',
 u'landfall']

# ANALYSIS STARTS HERE

In [197]:
hurName='TOMAS'
year = '2010'

In [198]:
hurdata = data[data['Name'].str.contains(hurName,na=False) & data['ISO_time'].str.contains(year,na=False)]

In [199]:
hurdata.iloc[0]

Serial_Num                     2010069S12188
Season                                  2010
Num                                       10
Basin                                     SP
Sub_basin                                 MM
Name                                   TOMAS
ISO_time                 2010-03-10 00:00:00
Nature                                    NR
Latitude                              -12.30
Longitude                            -172.20
Wind(WMO)                               20.0
Pres(WMO)                             1005.0
Center                                  nadi
Wind(WMO) Percentile                   4.063
Pres(WMO) Percentile                   1.388
Track_type                              main
Latitude_for_mapping                  -12.30
Longitude_for_mapping                -172.20
Current Basin                             SP
hurdat_atl_lat                        -999.0
hurdat_atl_lon                        -999.0
hurdat_atl_grade                      -999.0
hurdat_atl

In [200]:
hurdata.shape

(96, 200)

In [201]:
ntc=hurdata['Basin'].unique()

In [202]:
ntc

array([' SP', ' NA'], dtype=object)

In [203]:
hur1=hurdata.loc[hurdata['Basin']==ntc[1]]

In [204]:
hur1=hur1.dropna()

In [205]:
hur1[['Longitude','Latitude']]=hur1[['Longitude','Latitude']].apply(pd.to_numeric)

Check if we cross International Date Line (IDL)

In [206]:
sig=np.sign(hur1.Longitude.values)
sig1=sig[0]
m=sig != sig1

In [207]:
if sum(m)>0:
# adjust the lon values going from -180:180
        if sig1 > 0:
                hur1.loc[hur1.Longitude < 0, ['Longitude']] = hur1.Longitude+360.
        elif sig1 < 0:
                hur1.loc[hur1.Longitude > 0, ['Longitude']] = hur1.Longitude-360.



In [208]:
plt.plot(hur1['Longitude'],hur1['Latitude'],'o--')
mplleaflet.display()

In [209]:
idf=hur1.iloc[0].Serial_Num

In [210]:
filename=glob.glob('BestTrack/ibtracs/{}*'.format(idf))

In [211]:
dat=Dataset(filename[0])

In [212]:
dat.variables.keys()

[u'nobs_wmo',
 u'nobs',
 u'genesis_basin',
 u'num_basins',
 u'season',
 u'source',
 u'track_from_source',
 u'name_from_source',
 u'time_wmo',
 u'isotime_wmo',
 u'source_wmo',
 u'averaging_time',
 u'lat_wmo',
 u'lon_wmo',
 u'wind_wmo',
 u'pres_wmo',
 u'basin_wmo',
 u'sub_basin_wmo',
 u'nature_wmo',
 u'dist2land_wmo',
 u'landfall_wmo',
 u'pres_perc_wmo',
 u'wind_perc_wmo',
 u'time',
 u'isotime',
 u'lat_for_mapping',
 u'lon_for_mapping',
 u'wind_for_mapping',
 u'pres_for_mapping',
 u'nature_for_mapping',
 u'dist2land',
 u'landfall',
 u'lat_from_source',
 u'lon_from_source',
 u'wind_from_source',
 u'wind_avg_period',
 u'wind_perc_from_source',
 u'pres_from_source',
 u'pres_perc_from_source',
 u'basin',
 u'sub_basin',
 u'pos_stdev',
 u'pos_num',
 u'pos_flag',
 u'wind_quality',
 u'wind_stdev',
 u'wind_condition',
 u'wind_num',
 u'wind_flag',
 u'wind_maximum',
 u'wind_minimum',
 u'wind_median',
 u'pres_quality',
 u'pres_stdev',
 u'pres_condition',
 u'pres_flag',
 u'pres_num',
 u'pres_maximum'

In [ ]:
# wradii=dat['jtwc_sh_wrad'][:13]

In [213]:
wradiinp=np.array(wradii)

In [214]:
w34= wradiinp[:,0,:]

In [215]:
w34ne=w34[:,0]
w34nw=w34[:,1]
w34se=w34[:,2]
w34sw=w34[:,3]

In [216]:
w50=wradiinp[:,1,:]

In [217]:
w50ne=w50[:,0]
w50nw=w50[:,1]
w50se=w50[:,2]
w50sw=w50[:,3]

In [218]:
w64=wradiinp[:,2,:]

In [219]:
w64ne=w64[:,0]
w64nw=w64[:,1]
w64se=w64[:,2]
w64sw=w64[:,3]

In [220]:
isot=dat['isotime'][:13]

In [221]:
isot1=[''.join(x) for x in isot]

In [222]:
time=[re.sub(r'[-: ]+','',x)[:-4] for x in isot1]

In [226]:
lat=dat['atcf_lat'][:13]
lon=dat['atcf_lon'][:13]
pcenter=dat['atcf_pres'][:13]
vmaxh=dat['atcf_wind'][:13] # 10 minute wind in Knots
rmaxh=dat['atcf_rmw'][:13]*nm2m

In [227]:
dic={'t':time, 'lat':lat,'lon':lon,'pcenter':pcenter,'vmax':vmaxh, 'rmax':rmaxh, '34ne': w34ne, '34nw': w34nw, '34se':w34se, '34sw':w34sw, '50ne': w50ne, '50nw': w50nw, '50se':w50se, '50sw':w50sw,'64ne': w64ne, '64nw': w64nw, '64se':w64se, '64sw':w64sw} 

In [228]:
inpData=pd.DataFrame(dic)

In [229]:
inpData=inpData.apply(pd.to_numeric)

In [230]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,9.0,-53.700001,1006,222240.0,2010102906,30
1,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,9.8,-55.299999,1003,166680.0,2010102912,35
2,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,10.8,-56.799999,998,55560.0,2010102918,45
3,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,11.9,-57.799999,999,55560.0,2010103000,55
4,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,12.7,-58.900002,997,55560.0,2010103006,60
5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,13.1,-60.099998,993,37040.0,2010103012,65
6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,13.3,-61.000000,990,37040.0,2010103018,80
7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,13.5,-61.700001,982,37040.0,2010103100,85
8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,13.8,-62.400002,983,27780.0,2010103106,85
9,30,30,30,25,-1,-1,-1,-1,-1,-1,-1,-1,14.0,-63.299999,988,27780.0,2010103112,80


In [231]:
inpData.loc[inpData['lat'] == 9.969210e+36, 'lat'] = np.nan
inpData.loc[inpData['lon'] == 9.969210e+36, 'lon'] = np.nan
inpData.loc[inpData['pcenter'] == -32767., 'pcenter'] = np.nan
inpData.loc[inpData['rmax'] == -60684484., 'rmax'] = np.nan
inpData.loc[inpData['vmax'] == -32767, 'vmax'] = np.nan
inpData.loc[inpData['34ne'] == -32767, '34ne']=np.nan
inpData.loc[inpData['34nw'] == -32767, '34nw']=np.nan
inpData.loc[inpData['34se'] == -32767, '34se']=np.nan
inpData.loc[inpData['34sw'] == -32767, '34sw']=np.nan
inpData.loc[inpData['50ne'] == -32767, '50ne']=np.nan
inpData.loc[inpData['50nw'] == -32767, '50nw']=np.nan
inpData.loc[inpData['50se'] == -32767, '50se']=np.nan
inpData.loc[inpData['50sw'] == -32767, '50sw']=np.nan
inpData.loc[inpData['64ne'] == -32767, '64ne']=np.nan
inpData.loc[inpData['64nw'] == -32767, '64nw']=np.nan
inpData.loc[inpData['64se'] == -32767, '64se']=np.nan
inpData.loc[inpData['64sw'] == -32767, '64sw']=np.nan

In [232]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,-53.700001,1006.0,222240.0,2010102906,30.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.8,-55.299999,1003.0,166680.0,2010102912,35.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.8,-56.799999,998.0,55560.0,2010102918,45.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.9,-57.799999,999.0,55560.0,2010103000,55.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7,-58.900002,997.0,55560.0,2010103006,60.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13.1,-60.099998,993.0,37040.0,2010103012,65.0
6,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13.3,-61.000000,990.0,37040.0,2010103018,80.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13.5,-61.700001,982.0,37040.0,2010103100,85.0
8,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13.8,-62.400002,983.0,27780.0,2010103106,85.0
9,30.0,30.0,30.0,25.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14.0,-63.299999,988.0,27780.0,2010103112,80.0


In [233]:
inpData.loc[inpData['34ne'] == -1, '34ne']=0
inpData.loc[inpData['34nw'] == -1, '34nw']=0
inpData.loc[inpData['34se'] == -1, '34se']=0
inpData.loc[inpData['34sw'] == -1, '34sw']=0
inpData.loc[inpData['50ne'] == -1, '50ne']=0
inpData.loc[inpData['50nw'] == -1, '50nw']=0
inpData.loc[inpData['50se'] == -1, '50se']=0
inpData.loc[inpData['50sw'] == -1, '50sw']=0
inpData.loc[inpData['64ne'] == -1, '64ne']=0
inpData.loc[inpData['64nw'] == -1, '64nw']=0
inpData.loc[inpData['64se'] == -1, '64se']=0
inpData.loc[inpData['64sw'] == -1, '64sw']=0

In [234]:
inpData=inpData.dropna()

In [235]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.1,-60.099998,993.0,37040.0,2010103012,65.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.3,-61.000000,990.0,37040.0,2010103018,80.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.5,-61.700001,982.0,37040.0,2010103100,85.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.8,-62.400002,983.0,27780.0,2010103106,85.0
9,30.0,30.0,30.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,-63.299999,988.0,27780.0,2010103112,80.0
10,70.0,60.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,14.2,-64.400002,994.0,27780.0,2010103118,65.0
11,70.0,60.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,14.1,-65.800003,997.0,55560.0,2010110100,55.0
12,70.0,65.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,13.9,-67.099998,1003.0,55560.0,2010110106,45.0


In [236]:
dph=pn-inpData.pcenter*100.

In [237]:
dph

5     1700.0
6     2000.0
7     2800.0
8     2700.0
9     2200.0
10    1600.0
11    1300.0
12     700.0
Name: pcenter, dtype: float64

In [238]:
inpData=inpData.assign(dp=dph)

In [240]:
inpData.iloc[3]

34ne       0.000000e+00
34nw       0.000000e+00
34se       0.000000e+00
34sw       0.000000e+00
50ne       0.000000e+00
50nw       0.000000e+00
50se       0.000000e+00
50sw       0.000000e+00
64ne       0.000000e+00
64nw       0.000000e+00
64se       0.000000e+00
64sw       0.000000e+00
lat        1.380000e+01
lon       -6.240000e+01
pcenter    9.830000e+02
rmax       2.778000e+04
t          2.010103e+09
vmax       8.500000e+01
dp         2.700000e+03
Name: 8, dtype: float64

### create inpData file

In [132]:
inpData['t']=pd.to_datetime(inpData['t'],format='%Y%m%d%H')

In [133]:
inpData.head()

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.5,168.899994,1004.0,92600.0,2015-03-08 12:00:00,25.0,600.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.8,169.199997,1000.0,92600.0,2015-03-08 18:00:00,30.0,1000.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.1,169.600006,1000.0,92600.0,2015-03-09 00:00:00,30.0,1000.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,170.100006,996.0,92600.0,2015-03-09 06:00:00,35.0,1400.0
9,30.0,30.0,30.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,170.600006,989.0,55560.0,2015-03-09 12:00:00,45.0,2100.0


In [136]:
inpData.t.iloc[0]

Timestamp('2015-03-08 12:00:00')

In [137]:
inpData['time']=inpData['t']-inpData.t.iloc[0]

In [139]:
inpData['time']=inpData['time'] / pd.Timedelta('1 hour')

In [141]:
inpData=inpData.set_index('time')
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
time,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.5,168.899994,1004.0,92600.0,2015-03-08 12:00:00,25.0,600.0
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.8,169.199997,1000.0,92600.0,2015-03-08 18:00:00,30.0,1000.0
12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.1,169.600006,1000.0,92600.0,2015-03-09 00:00:00,30.0,1000.0
18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,170.100006,996.0,92600.0,2015-03-09 06:00:00,35.0,1400.0
24.0,30.0,30.0,30.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,170.600006,989.0,55560.0,2015-03-09 12:00:00,45.0,2100.0
30.0,70.0,60.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,-9.3,170.500000,982.0,37040.0,2015-03-09 18:00:00,55.0,2800.0
36.0,70.0,60.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,-9.7,170.500000,978.0,37040.0,2015-03-10 00:00:00,60.0,3200.0
42.0,70.0,65.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,-10.2,170.500000,974.0,37040.0,2015-03-10 06:00:00,65.0,3600.0


## convert to SI

In [142]:
inpData['vmax']=inpData['vmax']*kt2ms

In [143]:
inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]=inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]*nm2m

In [147]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
time,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.5,168.899994,1004.0,92600.0,2015-03-08 12:00:00,12.861111,600.0
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.8,169.199997,1000.0,92600.0,2015-03-08 18:00:00,15.433333,1000.0
12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.1,169.600006,1000.0,92600.0,2015-03-09 00:00:00,15.433333,1000.0
18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,170.100006,996.0,92600.0,2015-03-09 06:00:00,18.005556,1400.0
24.0,55560.0,55560.0,55560.0,46300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,170.600006,989.0,55560.0,2015-03-09 12:00:00,23.150000,2100.0
30.0,129640.0,111120.0,120380.0,138900.0,46300.0,37040.0,37040.0,46300.0,0.0,0.0,0.0,0.0,-9.3,170.500000,982.0,37040.0,2015-03-09 18:00:00,28.294444,2800.0
36.0,129640.0,111120.0,120380.0,138900.0,46300.0,37040.0,37040.0,46300.0,0.0,0.0,0.0,0.0,-9.7,170.500000,978.0,37040.0,2015-03-10 00:00:00,30.866667,3200.0
42.0,129640.0,120380.0,120380.0,138900.0,46300.0,37040.0,37040.0,46300.0,0.0,0.0,0.0,0.0,-10.2,170.500000,974.0,37040.0,2015-03-10 06:00:00,33.438889,3600.0


## save inpData file

In [150]:
column_order=['lat','lon','dp','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [152]:
header=['lat','long','dp','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [153]:
inpData.to_csv('tmp/inpDataBT.txt',index=True, columns=column_order, sep='\t', header=header)

### create bulInfo.txt file

In [71]:
tt=pd.to_datetime(isot1[0])

In [72]:
tt=datetime.datetime.strftime(tt,'%d %b %Y %H:%M:%S')

In [73]:
dic0={'advNo':[1],'tShift':[0],'$date':tt,'land':[1],'notes':[0]}

In [74]:
bul=pd.DataFrame.from_dict(dic0)

In [75]:
bul

,$date,advNo,land,notes,tShift
0,07 Mar 2015 06:00:00,1,1,0,0


In [76]:
bul.to_csv('tmp/bulInfo.txt',index=False, columns=['advNo','tShift','$date','land','notes'], sep='\t')

### create info.xml file

In [79]:
import xml.etree.ElementTree as et 
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = et.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [107]:
info = et.Element('setexp')
et.SubElement(info, 'source').text = 'Tropical Cyclone Bulletin through GDACS/PDC'
et.SubElement(info, 'hurName').text = hurName
et.SubElement(info, 'hurId').text = hurName
et.SubElement(info, 'basin').text = ntc[0].strip()
et.SubElement(info, 'bulNo').text = '1'
et.SubElement(info, 'bulDate').text = tt
et.SubElement(info, 'n').text = '100000'
et.SubElement(info, 'fk').text = '0.81'
et.SubElement(info, 'stormsurge').text = '0'
et.SubElement(info, 'timefactor').text = '1'
et.SubElement(info, 'landfall').text = '1'

In [116]:
xmlf = minidom.parseString(prettify(info))

In [117]:
with open('tmp/info.xml','w') as f:
      xmlf.writexml(f)

The next step is to substract translational and coriolis velocity, see [Subtract translational and Coriolis velocity.ipynb](./Subtract translational and Coriolis velocity.ipynb).